**Connect google drive**

In [3]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Sun Dec 11 09:31:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    26W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
'Colab Notebooks'  'My Drive'   yolov3


**1) Clone the Darknet**



In [5]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15502, done.
remote: Total 15502 (delta 0), reused 0 (delta 0), pack-reused 15502
Receiving objects: 100% (15502/15502), 14.16 MiB | 18.13 MiB/s, done.
Resolving deltas: 100% (10405/10405), done.


**2) Compile Darknet using Nvidia GPU**


In [6]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [7]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [8]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [9]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [10]:
!echo "Koala" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [11]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-12-11 09:32:43--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  16.9MB/s    in 13s     

2022-12-11 09:32:56 (12.1 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [12]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/25.txt         
  inflating: data/obj/26.jpg         
  inflating: data/obj/26.txt         
  inflating: data/obj/27.jpg         
  inflating: data/obj/27.txt         
  inflating: data/obj/28.jpg         
  inflating: data/obj/28.txt         
  inflating: data/obj/29.jpg         
  inflating: data/obj/29.txt         
  inflating: data/obj/30.jpg         
  inflating: data/obj/30.txt         
  inflating: data/obj/31.jpg         
  inflating: data/obj/31.txt         
  inflating: data/obj/32.jpg         
  inflating: data/obj/32.txt         
  inflating: data/obj/33.jpg         
  inflating: data/obj/33.txt         
  inflating: data/obj/34.jpg         
  inflating: data/obj/34.txt         
  inflating: data/obj/35.jpg         
  inflating: data/obj/35.txt         
  inflating: data/obj/36.jpg         
  inflating: data/obj/36.txt         
  inflating: data/obj/37.jpg         
  inflating: data/obj/37.txt         
  inflating: 

In [13]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

15 0.477679 0.491071 0.910714 0.946429

['15', '0.477679', '0.491071', '0.910714', '0.946429']
0 data/obj/3.txt
0 0.477679 0.491071 0.910714 0.946429
15 0.475446 0.500000 0.933036 0.964286

['15', '0.475446', '0.500000', '0.933036', '0.964286']
1 data/obj/4.txt
0 0.475446 0.500000 0.933036 0.964286
15 0.504464 0.506696 0.991071 0.986607

['15', '0.504464', '0.506696', '0.991071', '0.986607']
2 data/obj/106.txt
0 0.504464 0.506696 0.991071 0.986607
15 0.529018 0.493304 0.941964 0.950893

['15', '0.529018', '0.493304', '0.941964', '0.950893']
3 data/obj/36.txt
0 0.529018 0.493304 0.941964 0.950893
15 0.558036 0.524554 0.848214 0.933036

['15', '0.558036', '0.524554', '0.848214', '0.933036']
4 data/obj/22.txt
0 0.558036 0.524554 0.848214 0.933036
15 0.486607 0.486607 0.946429 0.955357

['15', '0.486607', '0.486607', '0.946429', '0.955357']
5 data/obj/10.txt
0 0.486607 0.486607 0.946429 0.955357
15 0.495536 0.486607 0.955357 0.946429

['15', '0.495536', '0.486607', '0.955357', '0.946429']


In [14]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/42.jpg', 'data/obj/104.jpg', 'data/obj/44.jpg', 'data/obj/33.jpg', 'data/obj/11.jpg', 'data/obj/31.jpg', 'data/obj/51.jpg', 'data/obj/14.jpg', 'data/obj/30.jpg', 'data/obj/17.jpg', 'data/obj/19.jpg', 'data/obj/60.jpg', 'data/obj/4.jpg', 'data/obj/76.jpg', 'data/obj/88.jpg', 'data/obj/71.jpg', 'data/obj/67.jpg', 'data/obj/105.jpg', 'data/obj/18.jpg', 'data/obj/48.jpg', 'data/obj/34.jpg', 'data/obj/54.jpg', 'data/obj/98.jpg', 'data/obj/87.jpg', 'data/obj/106.jpg', 'data/obj/13.jpg', 'data/obj/49.jpg', 'data/obj/85.jpg', 'data/obj/37.jpg', 'data/obj/61.jpg', 'data/obj/94.jpg', 'data/obj/2.jpg', 'data/obj/90.jpg', 'data/obj/3.jpg', 'data/obj/107.jpg', 'data/obj/97.jpg', 'data/obj/77.jpg', 'data/obj/96.jpg', 'data/obj/53.jpg', 'data/obj/6.jpg', 'data/obj/63.jpg', 'data/obj/110.jpg', 'data/obj/72.jpg', 'data/obj/23.jpg', 'data/obj/92.jpg', 'data/obj/50.jpg', 'data/obj/80.jpg', 'data/obj/12.jpg', 'data/obj/7.jpg', 'data/obj/103.jpg', 'data/obj/39.jpg', 'data/obj/35.jpg', 'data/obj/

In [15]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [16]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.795174, iou_loss = 0.000000, total_loss = 0.795174 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.751787, iou_loss = 0.000000, total_loss = 0.751787 
 total_bbox = 13943, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.440319), count: 4, class_loss = 2.046178, iou_loss = 0.823643, total_loss = 2.869821 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.827060, iou_loss = 0.000000, total_loss = 0.827060 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.812789, iou_loss = 0.000000, total_loss = 0.812789 
 total_bbox = 13947, rewritten_bbox = 0.000000 % 
v3 (mse